In [53]:
import torch
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [4]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [5]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [6]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.1808,  0.3185, -0.9667],
        [ 0.6865,  0.7698, -2.9489]], requires_grad=True)
tensor([-0.2655,  0.3848], requires_grad=True)


In [7]:
def model(x):
    return x @ w.t() + b

In [8]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ -7.2921, -24.7289],
        [-17.6489, -58.1331],
        [  2.0812,  -7.7759],
        [ -3.8937,  -5.6031],
        [-24.8784, -84.7701]], grad_fn=<AddBackward0>)


In [9]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [10]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(14188.2949, grad_fn=<DivBackward0>)


In [11]:
# Compute gradients
loss.backward()

In [12]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.1808,  0.3185, -0.9667],
        [ 0.6865,  0.7698, -2.9489]], requires_grad=True)
tensor([[ -7046.8149,  -8395.7100,  -5145.1870],
        [-10409.8965, -12121.6777,  -7652.6177]])


In [32]:
# Train for 100 epochs
for i in range(100000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [33]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(0.5137, grad_fn=<DivBackward0>)


In [34]:
print(preds)

tensor([[ 57.1029,  70.3431],
        [ 82.2459, 100.6488],
        [118.7032, 132.9530],
        [ 21.0892,  37.0108],
        [101.9105, 119.1434]], grad_fn=<AddBackward0>)


In [45]:
model = nn.Linear(3, 2)

In [51]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [52]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [47]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [48]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [141]:
fit(100, model, F.mse_loss, torch.optim.SGD(model.parameters(), lr=1e-5), train_dl)

Epoch [10/100], Loss: 0.5111
Epoch [20/100], Loss: 0.5111
Epoch [30/100], Loss: 0.5111
Epoch [40/100], Loss: 0.5111
Epoch [50/100], Loss: 0.5111
Epoch [60/100], Loss: 0.5111
Epoch [70/100], Loss: 0.5111
Epoch [80/100], Loss: 0.5111
Epoch [90/100], Loss: 0.5111
Epoch [100/100], Loss: 0.5111


In [144]:
preds = model(inputs)
preds

tensor([[ 57.0862,  70.3293],
        [ 82.2568, 100.6574],
        [118.7050, 132.9552],
        [ 21.0912,  37.0126],
        [101.9086, 119.1412],
        [ 55.8417,  69.2317],
        [ 82.0994, 100.7727],
        [118.9990, 133.5617],
        [ 22.3358,  38.1102],
        [102.9958, 120.3540],
        [ 56.9289,  70.4446],
        [ 81.0122,  99.5598],
        [118.8623, 132.8400],
        [ 20.0041,  35.7997],
        [103.1531, 120.2388]], grad_fn=<AddmmBackward0>)

In [143]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [153]:
model(torch.tensor([[75, 64, 44.]]))

tensor([[54.4398, 68.2494]], grad_fn=<AddmmBackward0>)